# 05. LLMベースの軽量マルチエージェント（Planner/Coder/Critic）

このノートブックでは、同じ3役構成を **LLM実行** に切り替えて、最小構成で動作確認します。


In [ ]:
# 編集禁止セル
import os
import sys
import torch

repo_path = '/content/llm_lab'
if os.path.exists(repo_path):
    !rm -rf {repo_path}
!git clone -b ai_agent https://github.com/akio-kobayashi/llm_lab.git {repo_path}
os.chdir(repo_path)

!pip install -q -U transformers accelerate bitsandbytes

if 'src' not in sys.path:
    sys.path.append(os.path.abspath('src'))

from src.common import load_llm
from src.agent_core import LLMPlannerCoderCriticAgent

print('セットアップ完了')



In [ ]:
# 編集禁止セル
model, tokenizer = load_llm(use_4bit=True)


def llm_chat(system_prompt: str, user_prompt: str, max_new_tokens: int = 512, temperature: float = 0.2):
    """安定動作用チャット関数。空出力時はフォールバック生成を行う。"""
    try:
        # japanese-stablelm向けに指示形式へ整形
        prompt = (
            "以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n"
            f"### システム:\n{system_prompt}\n\n"
            f"### 指示:\n{user_prompt}\n\n"
            "### 応答:\n"
        )

        inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=1024)
        device = next(model.parameters()).device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        prompt_len = inputs['input_ids'].shape[1]

        pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

        if hasattr(model, 'config'):
            model.config.use_cache = False
        if hasattr(model, 'generation_config'):
            model.generation_config.use_cache = False

        # 1st pass: greedy（EOS停止を無効化して空出力を避ける）
        with torch.inference_mode():
            gen_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                min_new_tokens=16,
                do_sample=False,
                use_cache=False,
                pad_token_id=pad_token_id,
                eos_token_id=None,
                repetition_penalty=1.05,
                remove_invalid_values=True,
                renormalize_logits=True,
            )

        new_ids = gen_ids[0][prompt_len:]
        text = tokenizer.decode(new_ids, skip_special_tokens=True).strip()

        # 2nd pass: sampling fallback
        if not text:
            with torch.inference_mode():
                gen_ids = model.generate(
                    **inputs,
                    max_new_tokens=max_new_tokens,
                    min_new_tokens=24,
                    do_sample=True,
                    temperature=max(temperature, 0.7),
                    top_p=0.9,
                    use_cache=False,
                    pad_token_id=pad_token_id,
                    eos_token_id=None,
                    repetition_penalty=1.05,
                    remove_invalid_values=True,
                    renormalize_logits=True,
                )
            new_ids = gen_ids[0][prompt_len:]
            text = tokenizer.decode(new_ids, skip_special_tokens=True).strip()

        if not text:
            text = tokenizer.decode(new_ids, skip_special_tokens=False).strip()

        return text if text else '[Empty output]'
    except Exception as e:
        return f'Error: {e}'


agent = LLMPlannerCoderCriticAgent(llm_chat)

queries = [
    'Pythonで数値を2倍する関数の最小コードを書いて。',
    '学習ロードマップを3ステップで提案して。',
    '2026年2月の最新アニメ情報を要約して。情報が足りない場合は前提を明示して。',
]

for q in queries:
    final_answer, full_log, steps = agent.run_pipeline(q)
    print('=' * 70)
    print('Q:', q)
    print('Final:', final_answer[:500])
    print('--- role log (head) ---')
    print(full_log[:1200])



## 演習

- `RoleConfig` を調整し、Planner/Coder/Critic の温度を変えて出力差を比較してください。
- Critic の指摘が弱い場合、`src/agent_core.py` の Critic system prompt を強化してください。
